In [1]:
from google.colab import drive
drive.mount('/content/drive')

data_loc = '/content/drive/My Drive/data_cs583_seq2seq'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Home 4: Build a seq2seq model for machine translation.

### Name: Weronika Zamlynny

### Task: Translate English to Polish

## 0. You will do the following:

1. Read and run my code.
2. Complete the code in Section 1.1 and Section 4.2.

    * Translation English to **German** is not acceptable!!! Try another language.
    
3. **Make improvements.** Directly modify the code in Section 3. Do at least one of the followings. By doing more, you will get up to 2 bonus scores to the total.

    * Bi-LSTM instead of LSTM
    
    * Multi-task learning (e.g., both English to French and English to Spanish)
    
    * Attention
    
4. Evaluate the translation using the BLEU score. 

    * Optional. Up to 2 bonus scores to the total.
    
5. Convert the notebook to .HTML file. 

    * The HTML file must contain the code and the output after execution.

6. Put the .HTML file in your own Github repo. 

7. Submit the link to the HTML file to Canvas

    * E.g., https://github.com/wangshusen/CS583A-2019Spring/blob/master/homework/HM4/seq2seq.html
    


## 1. Data preparation

1. Download data (e.g., "deu-eng.zip") from http://www.manythings.org/anki/
2. Unzip the .ZIP file.
3. Put the .TXT file (e.g., "deu.txt") in the directory "./Data/".

In [0]:
import re
import string
from unicodedata import normalize
import numpy

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

# THIS HAS AN ISSUE WITH POLISH CHARACTERS
def clean_data(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # THIS CAUSES WORDS TO LOSE IMPORTANT CHARACTERS!
            # normalize unicode characters
#             line = normalize('NFD', line).encode('ascii', 'ignore')          
#             line = line.decode('UTF-8')
            
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            
            # remove non-printable chars form each token
#             line = [re_print.sub('', w) for w in line]
            
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        
        cleaned.append(clean_pair)
    return numpy.array(cleaned)

### 1.1. Load and clean text


#### Fill the following blanks:

Wanted to use languages in the same family as polish (Slavic Languages), but many use the Cyrillic alphabet which would complicated encodings. Tried loading in Sloviakian, but the data set only had 600 entries, which limits its usefulness in training (Would have to remove many terms from the other languages to have a consistent set of translations. 

In [0]:
# e.g., filename = 'Data/deu.txt'
filename_pol = data_loc + '/pol.txt' # Polish
filename_fra = data_loc + '/fra.txt' # French
filename_spa = data_loc + '/spa.txt' # Spanish

In [0]:
# load dataset
doc_pol = load_doc(filename_pol)
doc_fra = load_doc(filename_fra)
doc_spa = load_doc(filename_spa)

# split into Language1-Language2 pairs
pairs_pol = to_pairs(doc_pol)
pairs_fra = to_pairs(doc_fra)
pairs_spa = to_pairs(doc_spa)

# clean sentences
clean_pairs_pol = clean_data(pairs_pol) #[0:n_train, :]
clean_pairs_fra = clean_data(pairs_fra)
clean_pairs_spa = clean_data(pairs_spa)

del doc_pol
del doc_fra
del doc_spa
del pairs_pol
del pairs_fra
del pairs_spa

In [5]:
type(clean_pairs_pol[0])

numpy.ndarray

In [6]:
print("Total number of data points (pol):", len(clean_pairs_pol))
print("Total number of data points (fra):", len(clean_pairs_fra)) # fra
print("Total number of data points (spa):", len(clean_pairs_spa)) # spa

Total number of data points (pol): 37426
Total number of data points (fra): 167130
Total number of data points (spa): 120614


In [7]:
for i in range(3000, 3010):
    print('[' + clean_pairs_pol[i, 0] + '] => [' + clean_pairs_pol[i, 1] + ']')

[please sit down] => [usiądź proszę]
[please sit down] => [proszę usiądź]
[please tell me] => [proszę powiedz mi]
[raise your hand] => [podnieś rękę]
[remove your hat] => [zdejmij kapelusz]
[school bores me] => [szkoła mnie nudzi]
[school bores me] => [nudzę się w szkole]
[see you about] => [do zobaczenia około siódmej]
[sharks eat fish] => [rekiny jedzą ryby]
[she admired him] => [ona go podziwiała]


In [8]:
for i in range(3000, 3010):
    print('[' + clean_pairs_fra[i, 0] + '] => [' + clean_pairs_fra[i, 1] + ']')

[whose is it] => [à qui estce]
[whose is it] => [cest à qui]
[will it fit] => [cela iratil]
[will it fit] => [cela passeratil]
[will it fit] => [cela sadapteratil]
[will you go] => [irezvous]
[will you go] => [irastu]
[will you go] => [ty rendrastu]
[will you go] => [vous y rendrezvous]
[work slowly] => [travaille lentement]


In [9]:
for i in range(3000, 3010):
    print('[' + clean_pairs_spa[i, 0] + '] => [' + clean_pairs_spa[i, 1] + ']')

[im prepared] => [estoy preparada]
[im rational] => [soy racional]
[im relieved] => [me siento aliviado]
[im restless] => [soy inquieto]
[im restless] => [estoy desasosegado]
[im restless] => [soy intranquilo]
[im shopping] => [estoy comprando]
[im sleeping] => [estoy durmiendo]
[im so alone] => [estoy tan solo]
[im so happy] => [soy tan feliz]


In [0]:
# # Convert the other languages to dictionaries to easily check for translations
# # this can't handle mutliple definitions...
dict_fra = {pair[0]: pair[1] for pair in clean_pairs_fra}
dict_spa = {pair[0]: pair[1] for pair in clean_pairs_spa}

In [0]:
# Create subsets that contain only overlap between all languages
final_pairs_pol = []
final_pairs_fra = []
final_pairs_spa = []

for i in range(len(clean_pairs_pol)):
  key = clean_pairs_pol[i,0]
  if key in dict_fra and key in dict_spa:
    # Other definition exists use it!
    final_pairs_pol.append(clean_pairs_pol[i])
    final_pairs_fra.append([key, dict_fra[key]])
    final_pairs_spa.append([key, dict_spa[key]])
    
clean_pairs_pol = numpy.array(final_pairs_pol)
clean_pairs_fra = numpy.array(final_pairs_fra)
clean_pairs_spa = numpy.array(final_pairs_spa)

del final_pairs_pol
del final_pairs_fra
del final_pairs_spa

del dict_fra
del dict_spa

In [12]:
len_data = len(clean_pairs_pol) # This is how much data we have
print("Final total number of data points (pol):", len_data)

Final total number of data points (pol): 8465


In [13]:
input_texts = clean_pairs_pol[:, 0]
target_texts = numpy.array(['\t' + text + '\n' for text in clean_pairs_pol[:, 1]])
target_texts_fra = numpy.array(['\t' + text + '\n' for text in clean_pairs_fra[:, 1]])
target_texts_spa = numpy.array(['\t' + text + '\n' for text in clean_pairs_spa[:, 1]])

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(target_texts.shape))
print('Length of target_texts_fra: ' + str(target_texts_fra.shape))
print('Length of target_texts_fra: ' + str(target_texts_spa.shape))

Length of input_texts:  (8465,)
Length of target_texts: (8465,)
Length of target_texts_fra: (8465,)
Length of target_texts_fra: (8465,)


In [0]:
del clean_pairs_pol
del clean_pairs_fra
del clean_pairs_spa

In [16]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length_pol = max(len(line) for line in target_texts)
max_decoder_seq_length_fra = max(len(line) for line in target_texts_fra)
max_decoder_seq_length_spa = max(len(line) for line in target_texts_spa)

max_decoder_seq_length = max([max_decoder_seq_length_pol, max_decoder_seq_length_fra, max_decoder_seq_length_spa])

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 235
max length of target sentences: 300


**Remark:** To this end, you have two lists of sentences: input_texts and target_texts

## 2. Text processing

### 2.1. Convert texts to sequences

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.

In [17]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# encode and pad sequences
def text2sequences(max_len, lines):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(lines)
    seqs = tokenizer.texts_to_sequences(lines)
    seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
    return seqs_pad, tokenizer.word_index


encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       target_texts)
# fra
decoder_input_seq_fra, target_token_index_fra = text2sequences(max_decoder_seq_length, 
                                                       target_texts_fra)
# spa
decoder_input_seq_spa, target_token_index_spa = text2sequences(max_decoder_seq_length, 
                                                       target_texts_spa)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

print('shape of decoder_input_seq (fra): ' + str(decoder_input_seq_fra.shape))
print('shape of target_token_index (fra): ' + str(len(target_token_index_fra)))

print('shape of decoder_input_seq (spa): ' + str(decoder_input_seq_spa.shape))
print('shape of target_token_index (spa): ' + str(len(target_token_index_spa)))

Using TensorFlow backend.


shape of encoder_input_seq: (8465, 235)
shape of input_token_index: 28
shape of decoder_input_seq: (8465, 300)
shape of target_token_index: 38
shape of decoder_input_seq (fra): (8465, 300)
shape of target_token_index (fra): 43
shape of decoder_input_seq (spa): (8465, 300)
shape of target_token_index (spa): 38


In [18]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

num_decoder_tokens_fra = len(target_token_index_fra) + 1
num_decoder_tokens_spa = len(target_token_index_spa) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

print('num_decoder_tokens (fra): ' + str(num_decoder_tokens_fra))
print('num_decoder_tokens (spa): ' + str(num_decoder_tokens_spa))

num_encoder_tokens: 29
num_decoder_tokens: 39
num_decoder_tokens (fra): 44
num_decoder_tokens (spa): 39


**Remark:** To this end, the input language and target language texts are converted to 2 matrices. 

- Their number of rows are both n_train.
- Their number of columns are respective max_encoder_seq_length and max_decoder_seq_length.

The followings print a sentence and its representation as a sequence.

In [19]:
target_texts[100]

'\tja to zrobiłem\n'

In [20]:
decoder_input_seq[100, :]

array([10, 18,  4,  1, 12,  5,  1,  6, 17,  5, 24,  3, 21,  2,  9, 11,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

## 2.2. One-hot encode

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.
- It is represented by a $n\times t \times v$ tensor ($t$ is the number of unique chars) after the one-hot encoding.

In [21]:
from keras.utils import to_categorical

# one hot encode target sequence
def onehot_encode(sequences, max_len, vocab_size):
    n = len(sequences)
    data = numpy.zeros((n, max_len, vocab_size))
    for i in range(n):
        data[i, :, :] = to_categorical(sequences[i], num_classes=vocab_size)
    return data

encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

# decoder_input_data_fra = onehot_encode(decoder_input_seq_fra, max_decoder_seq_length, num_decoder_tokens_fra)
# decoder_input_data_spa = onehot_encode(decoder_input_seq_spa, max_decoder_seq_length, num_decoder_tokens_spa)


decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)
# fra
decoder_target_seq_fra = numpy.zeros(decoder_input_seq_fra.shape)
decoder_target_seq_fra[:, 0:-1] = decoder_input_seq_fra[:, 1:]
decoder_target_data_fra = onehot_encode(decoder_target_seq_fra, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens_fra)
# spa
decoder_target_seq_spa = numpy.zeros(decoder_input_seq_spa.shape)
decoder_target_seq_spa[:, 0:-1] = decoder_input_seq_spa[:, 1:]
decoder_target_data_spa = onehot_encode(decoder_target_seq_spa, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens_spa)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(8465, 235, 29)
(8465, 300, 39)


### Random train / valid / test split

In [0]:
# Create a random mask that can be used to split each part of data
msk = numpy.random.rand(len_data)

train_msk = (msk <= 0.6)
valid_msk = numpy.logical_and(msk > 0.6, msk <= 0.8)
test_msk = (msk > 0.8)

del msk

In [23]:
train_encoder_input_data = encoder_input_data[train_msk,:,:]
valid_encoder_input_data = encoder_input_data[valid_msk,:,:]
test_encoder_input_data = encoder_input_data[test_msk,:,:]

del encoder_input_data


train_decoder_input_data = decoder_input_data[train_msk,:,:]
valid_decoder_input_data = decoder_input_data[valid_msk,:,:]
test_decoder_input_data = decoder_input_data[test_msk,:,:]

# fra
# train_decoder_input_data_fra = decoder_input_data_fra[train_msk,:,:]
# valid_decoder_input_data_fra = decoder_input_data_fra[valid_msk,:,:]
# test_decoder_input_data_fra = decoder_input_data_fra[test_msk,:,:]
# # spa
# train_decoder_input_data_spa = decoder_input_data_spa[train_msk,:,:]
# valid_decoder_input_data_spa = decoder_input_data_spa[valid_msk,:,:]
# test_decoder_input_data_spa = decoder_input_data_spa[test_msk,:,:]


del decoder_input_data

train_decoder_target_data = decoder_target_data[train_msk,:,:]
valid_decoder_target_data = decoder_target_data[valid_msk,:,:]
test_decoder_target_data = decoder_target_data[test_msk,:,:]

# fra
train_decoder_target_data_fra = decoder_target_data_fra[train_msk,:,:]
valid_decoder_target_data_fra = decoder_target_data_fra[valid_msk,:,:]
test_decoder_target_data_fra = decoder_target_data_fra[test_msk,:,:]
# spa
train_decoder_target_data_spa = decoder_target_data_spa[train_msk,:,:]
valid_decoder_target_data_spa = decoder_target_data_spa[valid_msk,:,:]
test_decoder_target_data_spa = decoder_target_data_spa[test_msk,:,:]

del decoder_target_data

train_input_texts = input_texts[train_msk]
valid_input_texts = input_texts[valid_msk]
test_input_texts = input_texts[test_msk]

del input_texts

train_target_texts = target_texts[train_msk]
valid_target_texts = target_texts[valid_msk]
test_target_texts = target_texts[test_msk]

# fra
train_target_texts_fra = target_texts_fra[train_msk]
valid_target_texts_fra = target_texts_fra[valid_msk]
test_target_texts_fra = target_texts_fra[test_msk]
# spa
train_target_texts_spa = target_texts_spa[train_msk]
valid_target_texts_spa = target_texts_spa[valid_msk]
test_target_texts_spa = target_texts_spa[test_msk]

del target_texts

print('shape of train_encoder_input_seq', train_encoder_input_data.shape)
print('shape of valid_encoder_input_seq', valid_encoder_input_data.shape)
print('shape of test_encoder_input_seq', test_encoder_input_data.shape)

print('shape of train_decoder_input_seq', train_decoder_input_data.shape)
print('shape of valid_decoder_input_seq', valid_decoder_input_data.shape)
print('shape of test_decoder_input_seq', test_decoder_input_data.shape)

shape of train_encoder_input_seq (5099, 235, 29)
shape of valid_encoder_input_seq (1676, 235, 29)
shape of test_encoder_input_seq (1690, 235, 29)
shape of train_decoder_input_seq (5099, 300, 39)
shape of valid_decoder_input_seq (1676, 300, 39)
shape of test_decoder_input_seq (1690, 300, 39)


## 3. Build the networks (for training)

- Build encoder, decoder, and connect the two modules to get "model". 

- Fit the model on the bilingual data to train the parameters in the encoder and decoder.

### 3.1. Encoder network

- Input:  one-hot encode of the input language

- Return: 

    -- output (all the hidden states   $h_1, \cdots , h_t$) are always discarded
    
    -- the final hidden state  $h_t$
    
    -- the final conveyor belt $c_t$

In [24]:
from keras.layers import Input, LSTM, Bidirectional, Concatenate
from keras.models import Model

latent_dim = 256

# inputs of the encoder network
encoder_inputs = Input(shape=(None, num_encoder_tokens), 
                       name='encoder_inputs')

# # set the LSTM layer
encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, 
                                  dropout=0.5, name='encoder_lstm'))
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

state_h = Concatenate(name="state_h")([forward_h, backward_h])
state_c = Concatenate(name="state_c")([forward_c, backward_c])

# build the encoder network model
encoder_model = Model(inputs=encoder_inputs, 
                      outputs=[state_h, state_c],
                      name='encoder')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Print a summary and save the encoder network structure to "./encoder.pdf"

In [25]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(encoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=encoder_model, show_shapes=False,
    to_file= data_loc+'/encoder.pdf'
)

encoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, None, 29)     0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, 512), (None, 585728      encoder_inputs[0][0]             
__________________________________________________________________________________________________
state_h (Concatenate)           (None, 512)          0           bidirectional_1[0][1]            
                                                                 bidirectional_1[0][3]            
__________________________________________________________________________________________________
state_c (Concatenate)           (None, 512)          0           bidirectional_1[0][2]            
          

### 3.2. Decoder network

- Inputs:  

    -- one-hot encode of the target language
    
    -- The initial hidden state $h_t$ 
    
    -- The initial conveyor belt $c_t$ 

- Return: 

    -- output (all the hidden states) $h_1, \cdots , h_t$

    -- the final hidden state  $h_t$ (discarded in the training and used in the prediction)
    
    -- the final conveyor belt $c_t$ (discarded in the training and used in the prediction)

In [0]:
# from keras.layers import Input, LSTM, Dense
# from keras.models import Model

# # inputs of the decoder network
# decoder_input_h = Input(shape=(latent_dim*2,), name='decoder_input_h')
# decoder_input_c = Input(shape=(latent_dim*2,), name='decoder_input_c')
# decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# # set the LSTM layer
# decoder_lstm = LSTM(latent_dim*2, return_sequences=True, 
#                     return_state=True, dropout=0.5, name='decoder_lstm')
# decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_input_x, 
#                       initial_state=[decoder_input_h, decoder_input_c])

# # set the dense layer
# decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
# decoder_outputs = decoder_dense(decoder_lstm_outputs)

# # build the decoder network model
# decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
#                       outputs=[decoder_outputs, state_h, state_c],
#                       name='decoder')

In [0]:
# Multi Task Learning!
from keras.layers import Input, LSTM, Dense
from keras.models import Model

# inputs of the decoder network
decoder_input_h = Input(shape=(latent_dim*2,), name='decoder_input_h')
decoder_input_c = Input(shape=(latent_dim*2,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(latent_dim*2, return_sequences=True, 
                    return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_input_x, 
                      initial_state=[decoder_input_h, decoder_input_c])

# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# fra
decoder_dense_fra = Dense(num_decoder_tokens_fra, activation='softmax', name='decoder_dense_fra')
decoder_outputs_fra = decoder_dense_fra(decoder_lstm_outputs)
# spa
decoder_dense_spa = Dense(num_decoder_tokens_spa, activation='softmax', name='decoder_dense_spa')
decoder_outputs_spa = decoder_dense_spa(decoder_lstm_outputs)


# build the decoder network model
decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                      outputs=[decoder_outputs, decoder_outputs_fra, decoder_outputs_spa, state_h, state_c],
                      name='decoder')

Print a summary and save the encoder network structure to "./decoder.pdf"

In [28]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(decoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=decoder_model, show_shapes=False,
    to_file= data_loc + '/decoder.pdf'
)

decoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input_x (InputLayer)    (None, None, 39)     0                                            
__________________________________________________________________________________________________
decoder_input_h (InputLayer)    (None, 512)          0                                            
__________________________________________________________________________________________________
decoder_input_c (InputLayer)    (None, 512)          0                                            
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 512),  1130496     decoder_input_x[0][0]            
                                                                 decoder_input_h[0][0]            
          

### 3.3. Connect the encoder and decoder

In [0]:
# input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# connect encoder to decoder
encoder_final_states = encoder_model([encoder_input_x])
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=encoder_final_states)
decoder_pred = decoder_dense(decoder_lstm_output)
decoder_pred_fra = decoder_dense_fra(decoder_lstm_output)
decoder_pred_spa = decoder_dense_spa(decoder_lstm_output)

model = Model(inputs=[encoder_input_x, decoder_input_x], 
              outputs=[decoder_pred, decoder_pred_fra, decoder_pred_spa], 
              name='model_training')

In [30]:
print(state_h)
print(decoder_input_h)

Tensor("decoder_lstm/while/Exit_2:0", shape=(?, 512), dtype=float32)
Tensor("decoder_input_h:0", shape=(?, 512), dtype=float32)


In [31]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=model, show_shapes=False,
    to_file= data_loc + '/model_training.pdf'
)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input_x (InputLayer)    (None, None, 29)     0                                            
__________________________________________________________________________________________________
decoder_input_x (InputLayer)    (None, None, 39)     0                                            
__________________________________________________________________________________________________
encoder (Model)                 [(None, 512), (None, 585728      encoder_input_x[0][0]            
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 512),  1130496     decoder_input_x[0][0]            
                                                                 encoder[1][0]                    
          

### 3.5. Fit the model on the bilingual dataset

- encoder_input_data: one-hot encode of the input language

- decoder_input_data: one-hot encode of the input language

- decoder_target_data: labels (left shift of decoder_input_data)

- tune the hyper-parameters

- stop when the validation loss stop decreasing.

In [32]:
print('shape of encoder_input_data' + str(train_encoder_input_data.shape))
print('shape of decoder_input_data' + str(train_decoder_input_data.shape))
print('shape of decoder_target_data' + str(train_decoder_target_data.shape))

shape of encoder_input_data(5099, 235, 29)
shape of decoder_input_data(5099, 300, 39)
shape of decoder_target_data(5099, 300, 39)


In [33]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit([train_encoder_input_data, train_decoder_input_data],  # training data
          [train_decoder_target_data, train_decoder_target_data_fra, train_decoder_target_data_spa],                       # labels (left shift of the target sequences)
          batch_size=64, epochs=50, 
          validation_data=[[valid_encoder_input_data, valid_decoder_input_data],
                           [valid_decoder_target_data, valid_decoder_target_data_fra, valid_decoder_target_data_spa]])

model.save('seq2seq.h5')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 5099 samples, validate on 1676 samples
Epoch 1/50
5099/5099 [==============================] - 95s 19ms/step - loss: 1.8580 - decoder_dense_loss: 0.5931 - decoder_dense_fra_loss: 0.6671 - decoder_dense_spa_loss: 0.5978 - val_loss: 1.1553 - val_decoder_dense_loss: 0.3640 - val_decoder_dense_fra_loss: 0.4259 - val_decoder_dense_spa_loss: 0.3653
Epoch 2/50
5099/5099 [==============================] - 94s 18ms/step - loss: 1.2245 - decoder_dense_loss: 0.3904 - decoder_dense_fra_loss: 0.4440 - decoder_dense_spa_loss: 0.3901 - val_loss: 1.1313 - val_decoder_dense_loss: 0.3579 - val_decoder_dense_fra_loss: 0.4119 - val_decoder_dense_spa_loss: 0.3615
Epoch 3/50
5099/5099 [==============================] - 92s 18ms/step - loss: 1.0806 - decoder_dense_loss: 0.3387 - decoder_dense_fra_loss: 0.3931 - decoder_dense_spa_loss: 0.3488 - val_loss: 0.9798 - val_decoder_d

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer decoder_lstm was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'encoder/state_h/concat:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'encoder/state_c/concat:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


## 4. Make predictions


### 4.1. Translate English to XXX

1. Encoder read a sentence (source language) and output its final states, $h_t$ and $c_t$.
2. Take the [star] sign "\t" and the final state $h_t$ and $c_t$ as input and run the decoder.
3. Get the new states and predicted probability distribution.
4. sample a char from the predicted probability distribution
5. take the sampled char and the new states as input and repeat the process (stop if reach the [stop] sign "\n").

In [0]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [0]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = numpy.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, output_tokens_fra, output_tokens_spa, h, c = decoder_model.predict([target_seq] + states_value)

        # this line of code is greedy selection
        # try to use multinomial sampling instead (with temperature)
        sampled_token_index = numpy.argmax(output_tokens[0, -1, :])
        
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
 
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


In [36]:
for seq_index in range(100, 120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = valid_encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('English:       ', valid_input_texts[seq_index])
    print('Polish (true): ', valid_target_texts[seq_index][1:-1])
    print('Polish (pred): ', decoded_sentence[0:-1])

-
English:        it was night
Polish (true):  była noc
Polish (pred):  to jest telefonie
-
English:        its bedtime
Polish (true):  czas spać
Polish (pred):  to jest nie spodziny
-
English:        keep looking
Polish (true):  szukaj dalej
Polish (pred):  przestaj się
-
English:        keep singing
Polish (true):  śpiewaj nadal
Polish (pred):  przestaj się
-
English:        lets go now
Polish (true):  teraz chodźmy
Polish (pred):  zajamijmy
-
English:        meat please
Polish (true):  mięso proszę
Polish (pred):  pozyj mi się
-
English:        nobody knows
Polish (true):  nikt nie wie
Polish (pred):  nikt nie wienie
-
English:        she may come
Polish (true):  ona być może przyjdzie
Polish (pred):  ona jest pieniądzy
-
English:        she went out
Polish (true):  wyszła
Polish (pred):  ona miesz powiedział
-
English:        stop smoking
Polish (true):  przestań palić
Polish (pred):  przestań się pomóc
-
English:        stop the car
Polish (true):  zatrzymaj samochód
Polish (pred)

### 4.2. Translate an English sentence to the target language

1. Tokenization
2. One-hot encode
3. Translate

In [37]:
input_sentence = 'why is that'
input_sentence_polish = "dlaczego"

input_sequence_length = len(input_sentence);
input_sequence, input_token_index = text2sequences(input_sequence_length, [input_sentence]) # TODO

input_x = onehot_encode([input_sequence], input_sequence_length, num_encoder_tokens) # TODO

translated_sentence = decode_sequence([input_x]) # TODO

print('source sentence is: ' + input_sentence)
print('translated sentence is: ' + translated_sentence)

source sentence is: why is that
translated sentence is: przedzieliłem



## 5. Evaluate the translation using BLEU score

Reference: 
- https://machinelearningmastery.com/calculate-bleu-score-for-text-python/
- https://en.wikipedia.org/wiki/BLEU


**Hint:** Randomly partition the dataset to training, validation, and test. Evaluate the BLEU score using the test set.

In [38]:
from nltk.translate import bleu_score 
# has sentence_bleu and corpus_blue, use corpus on the test set
sentence_score = bleu_score.sentence_bleu(input_sentence_polish, translated_sentence)
print("Sentence Bleu:", sentence_score)

Sentence Bleu: 0.7311104457090247


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


### Translate the test set

In [0]:
test_translated = [decode_sequence(test_encoder_input_data[seq_index: seq_index + 1]) for seq_index in range(len(test_encoder_input_data))]

In [40]:
for seq_index in range(100, 105):
    input_seq = test_encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = test_translated[seq_index: seq_index + 1]
    print('-')
    print('English:       ', test_input_texts[seq_index])
    print('Polish (true): ', test_target_texts[seq_index][1:-1])
    print('Polish (pred): ', decoded_sentence[0])

-
English:        stand aside
Polish (true):  stań z boku
Polish (pred):  przestań się

-
English:        take a rest
Polish (true):  odpocznij
Polish (pred):  pozyjśmi

-
English:        thats true
Polish (true):  to prawda
Polish (pred):  to jest dom

-
English:        they smiled
Polish (true):  uśmiechnęli się
Polish (pred):  przyszli

-
English:        this stinks
Polish (true):  to śmierdzi
Polish (pred):  to jest piena



### Final Corpus Bleu Score

In [41]:
corpus_score = bleu_score.corpus_bleu(test_target_texts, test_translated)
print("Corpus Bleu:", corpus_score)

Corpus Bleu: 0.7853703102937719


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
